In [1]:
import pandas as pd
import numpy as np 
from os import path


In [2]:
DATA_DIR = "../../data"

In [3]:
chen_train = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_train_data.csv"), index_col=0)
chen_valid = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_valid_data.csv"), index_col=0)
chen_test = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_test_data.csv"), index_col=0)
chen_train.head()

,Antibody_ID,heavy,light,Y
2073,6aod,EVQLVQSGAEVKKPGASVKVSCKASGYTFTGYYMHWVRQAPGQGLE...,DIVMTKSPSSLSASVGDRVTITCRASQGIRNDLGWYQQKPGKAPKR...,0
1517,4yny,EVQLVESGGGLVQPGRSLKLSCAASGFTFSNYGMAWVRQTPTKGLE...,EFVLTQPNSVSTNLGSTVKLSCKRSTGNIGSNYVNWYQQHEGRSPT...,1
2025,5xcv,EVQLVESGGGLVQPGRSLKLSCAASGFTFSNYGMAWVRQTPTKGLE...,QFVLTQPNSVSTNLGSTVKLSCKRSTGNIGSNYVNWYQQHEGRSPT...,1
2070,6and,EVQLVESGGGLVQPGGSLRLSCAASGYEFSRSWMNWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRSSQSIVHSVGNTFLEWYQQKPG...,1
666,2xqy,QVQLQQPGAELVKPGASVKMSCKASGYSFTSYWMNWVKQRPGRGLE...,DIVLTQSPASLALSLGQRATISCRASKSVSTSGYSYMYWYQQKPGQ...,0


In [5]:
with open(path.join(DATA_DIR, "fairseq/heavy/input0/train"), "w") as dat:
    with open(path.join(DATA_DIR, "fairseq/heavy/label/train.label"), "w") as lab:
        for i, row in chen_train.iterrows():
            dat.write(row["heavy"] + '\n')
            lab.write(str(row["Y"]) + '\n')

In [10]:
with open(path.join(DATA_DIR, "fairseq/light/input0/train"), "w") as dat:
    with open(path.join(DATA_DIR, "fairseq/light/label/train.label"), "w") as lab:
        for i, row in chen_train.iterrows():
            dat.write(row["light"] + '\n')
            lab.write(str(row["Y"]) + '\n')

In [6]:
train_seqs = [row["heavy"] for i, row in chen_train.iterrows()]

In [7]:
with open(path.join(DATA_DIR, "fairseq/heavy/input0/valid"), "w") as dat:
    with open(path.join(DATA_DIR, "fairseq/heavy/label/valid"), "w") as lab:
        for i, row in chen_valid.iterrows():
            dat.write(row["heavy"] + '\n')
            lab.write(str(row["Y"]) + '\n')

In [11]:
with open(path.join(DATA_DIR, "fairseq/light/input0/valid"), "w") as dat:
    with open(path.join(DATA_DIR, "fairseq/light/label/valid"), "w") as lab:
        for i, row in chen_valid.iterrows():
            dat.write(row["light"] + '\n')
            lab.write(str(row["Y"]) + '\n')

In [10]:
valid_seqs = [row["heavy"] + row["light"] for i, row in chen_valid.iterrows()]

In [12]:
from fairseq.data import Dictionary
from collections import Counter

In [14]:
token_counter = Counter()
for record in train_seqs:
    token_counter.update(record)
    
dictionary = Dictionary()
for token, count in sorted(token_counter.items()):
    dictionary.add_symbol(token, count)
    
with open(path.join(DATA_DIR, "fairseq/heavy/dict.txt"), "w") as f:
    dictionary.save(f)

In [53]:
!fairseq-train ../../data/fairseq/heavy/ \
    --restore-file ../../../../BioPhi/biophi/humanization/methods/sapiens/models/v1/checkpoint_vh.pt \
    --reset-optimizer --reset-dataloader --reset-meters \
    --dataset-impl raw \
    --task sentence_prediction \
    --criterion sentence_prediction \
    --user-dir ../../../../BioPhi/biophi/humanization/methods/sapiens \
    --arch roberta_small \
    --max-tokens 8096 \
    --update-freq 4 \
    --lr 1e-4 \
    --optimizer adam \
    --lr-scheduler inverse_sqrt \
    --warmup-updates 16000 \
    --max-positions 144 \
    --skip-invalid-size-inputs-valid-test \
    --validate-interval-updates 5000 \
    --save-interval-updates 5000 \
    --save-dir ../../data/models/fairseq \
    --num-classes 2 \
    --shorten-method truncate


2022-02-02 10:02:29 | INFO | fairseq_cli.train | Namespace(activation_dropout=0.0, activation_fn='gelu', adam_betas='(0.9, 0.999)', adam_eps=1e-08, add_prev_output_tokens=False, all_gather_list_size=16384, arch='roberta_small', attention_dropout=0.1, batch_size=None, batch_size_valid=None, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', classification_head_name='sentence_classification_head', clip_norm=0.0, cpu=False, criterion='sentence_prediction', curriculum=0, data='../../data/fairseq/heavy/', data_buffer_size=10, dataset_impl='raw', ddp_backend='c10d', device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_num_procs=0, distributed_port=-1, distributed_rank=0, distributed_world_size=1, distributed_wrapper='DDP', dropout=0.1, empty_cache_freq=0, encoder_attention_heads=8, encoder_embed_dim=128, encoder_ffn_e

In [ ]:
!fairseq-train ../../data/fairseq/heavy/ \
    --restore-file ../../../../BioPhi/biophi/humanization/methods/sapiens/models/v1/checkpoint_vh.pt \
    --reset-optimizer --reset-dataloader --reset-meters \
    --dataset-impl raw \
    --task sentence_prediction \
    --criterion sentence_prediction \
    --classification-head-name di-pred \
    --user-dir ../../../../BioPhi/biophi/humanization/methods/sapiens \
    --arch roberta_small \
    --batch-size 256 \
    --update-freq 1 \
    --lr 1e-5 \
    --optimizer adam \
    --lr-scheduler inverse_sqrt \
    --warmup-updates 10000 \
    --max-epoch 2000 \
    --max-positions 144 \
    --save-dir ../../data/models/fairseq \
    --num-classes 2 \
    --shorten-method truncate \
    --validate-interval 1 \
    --save-interval 10

2022-02-28 18:43:43 | INFO | fairseq_cli.train | Namespace(activation_dropout=0.0, activation_fn='gelu', adam_betas='(0.9, 0.999)', adam_eps=1e-08, add_prev_output_tokens=False, all_gather_list_size=16384, arch='roberta_small', attention_dropout=0.1, batch_size=256, batch_size_valid=256, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', classification_head_name='di-pred', clip_norm=0.0, cpu=False, criterion='sentence_prediction', curriculum=0, data='../../data/fairseq/heavy/', data_buffer_size=10, dataset_impl='raw', ddp_backend='c10d', device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_num_procs=0, distributed_port=-1, distributed_rank=0, distributed_world_size=1, distributed_wrapper='DDP', dropout=0.1, empty_cache_freq=0, encoder_attention_heads=8, encoder_embed_dim=128, encoder_ffn_embed_dim=256, encoder_l

In [55]:
! fairseq-train --help

usage: fairseq-train [-h] [--no-progress-bar] [--log-interval LOG_INTERVAL]
                     [--log-format {json,none,simple,tqdm}]
                     [--tensorboard-logdir TENSORBOARD_LOGDIR] [--seed SEED]
                     [--cpu] [--tpu] [--bf16] [--memory-efficient-bf16]
                     [--fp16] [--memory-efficient-fp16]
                     [--fp16-no-flatten-grads]
                     [--fp16-init-scale FP16_INIT_SCALE]
                     [--fp16-scale-window FP16_SCALE_WINDOW]
                     [--fp16-scale-tolerance FP16_SCALE_TOLERANCE]
                     [--min-loss-scale MIN_LOSS_SCALE]
                     [--threshold-loss-scale THRESHOLD_LOSS_SCALE]
                     [--user-dir USER_DIR]
                     [--empty-cache-freq EMPTY_CACHE_FREQ]
                     [--all-gather-list-size ALL_GATHER_LIST_SIZE]
                     [--model-parallel-size MODEL_PARALLEL_SIZE]
                     [--checkpoint-suffix CHECKPOINT_SUFFIX]
          

In [5]:
from biophi.humanization.methods.sapiens.roberta import RoBERTa, seq2tokens

In [ ]:
roberta = RoBERTa.load(
    path.join(DATA_DIR, "models/fairseq"),
    'checkpoint_best.pt'
)

In [7]:
tokens = seq2tokens("ASDFGHKLPIYTRE", roberta.interface.task.target_dictionary, True)

In [8]:
tokens

tensor([ 0,  4, 19,  6,  8,  9, 10, 12, 13, 16, 11, 23, 20, 18,  7,  2])

In [57]:
model = roberta.interface.model

In [58]:
model.classification_heads

ModuleDict(
  (di-pred): RobertaClassificationHead(
    (dense): Linear(in_features=128, out_features=128, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (out_proj): Linear(in_features=128, out_features=2, bias=True)
  )
)

In [28]:
roberta.interface.predict("sentence_classification_head", tokens)

tensor([[-0.6514, -0.7367]], grad_fn=<LogSoftmaxBackward0>)

In [7]:
label_fn = lambda label: roberta.interface.task.label_dictionary.string(
    [label + roberta.interface.task.label_dictionary.nspecial]
)

In [31]:
pred = label_fn(roberta.interface.predict('sentence_classification_head', tokens).argmax().item())

In [32]:
pred

'0'

In [8]:
seq = chen_test.iloc[0]["heavy"]
chen_test.iloc[0]["Y"]

0

In [9]:
tokens1 = seq2tokens(seq, roberta.interface.task.target_dictionary, True)

In [10]:
pred = label_fn(roberta.interface.predict('di-pred', tokens1).argmax().item())
pred

'0'

In [11]:
def get_pred(seq):
    tokens = seq2tokens(seq, roberta.interface.task.target_dictionary, True)
    return label_fn(roberta.interface.predict('di-pred', tokens).argmax().item())

In [12]:
chen_test["predictions"] = chen_test["heavy"].apply(get_pred)

In [13]:
chen_test.head()

,Antibody_ID,heavy,light,Y,predictions
2372,6obd,EVQLVESGGGLVQPGGSLRLSCAASGFPFSNYWMNWVRQAPGKGLE...,DIVMTQTPLSLSVTPGQPASISCKSSQSLLYSNGKTYLNWVLQKPG...,0,0
359,1sm3,QVQLQESGGGLVQPGGSMKLSCVASGFTFSNYWMNWVRQSPEKGLE...,DIVVTQESALTTSPGETVTLTCRSSTGAVTTSNYANWVQEKPDHLF...,0,0
1539,5a2i,QVQLQESGGGLVQPGGSMKLSCVASGFTFSNYWMNWVRQSPEKGLE...,DIVVTQESALTTSPGETVTLTCRSSTGAVTTSNYANWVQEKPDHLF...,0,0
1112,4f33,EVQLQQSGPELEKPGASVKISCKASGYSFTGYTMNWVKQSHGKSLE...,DIELTQSPAIMSASPGEKVTMTCSASSSVSYMHWYQQKSGTSPKRW...,1,0
314,1q1j,EVQLVESGGGLVKPGGSLRLTCVASGFTFSDVWLNWVRQAPGKGLE...,QSVLTQPPSVSAAPGQKVTISCSGSSSNIGNNYVLWYQQFPGTAPK...,0,0


In [14]:
chen_test["predictions"].value_counts()

0    119
Name: predictions, dtype: int64